# Практическое задание 3
# Генерация bash команды по текстовому запросу
## курс "Математические методы анализа текстов"
### ФИО:

### Постановка задачи

В этом задании вы построите систему, выдающую пользователю последовательность утилит командной строки linux (с нужными флагами) по его текстовому запросу. Вам дан набор пар текстовый запрос - команда на выходе.

Решение этого задания будет построено на encoder-decoder архитектуре и модели transformer.


### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
* pytorch
* transformers
* sentencepiece (bpe токенизация)
* clai utils (скачать с гитхаба отсюда https://github.com/IBM/clai/tree/nlc2cmd/utils)


### Данные

В качестве обучающей выборке используются данные, сгенерированные автоматически по запросам с сайта stack overflow. В качестве тестовых данных используются пары запросов, размеченные асессорами.

Данные можно скачать по ссылке: https://drive.google.com/file/d/1n457AAgrMwd5VbT6mGZ_rws3g2wwdEfX/view?usp=sharing

### Метрика качества

Ваш алгоритм должен выдавать пять вариантов ответа для каждого запроса.
Для упрощения задачи метрика качества будет учитывать утилиты и флаги ответа, но не учитывать подставленные значения. Пусть $\{ u_1, \ldots, u_T \}$, $\{ f_1, \ldots, f_T \}$ --- список утилит и множества их флагов ответа алгоритма, $\{v_1, \ldots, v_T \}$, $\{ \phi_1, \ldots, \phi_T \}$ --- список утилит и множества их флагов эталонного ответа. Если ответы отличаются по длине, они дополняются `None` утилитой.

$$ S = \frac{1}{T} \sum_{i=1}^{T} \left(\mathbb{I}[u_i = v_i]\left( 1 + \frac{1}{2}s(f_i, \phi_i)\right) - 1\right)$$

$$ s(f, \phi) = 1 + \frac{2 |f \cap \phi| - |f \cup \phi|}{\max(|f|, |\phi|)} $$

Метрика учитывает, что предсказать правильную утилиту важнее чем правильный флаг. При этом порядок флагов не важен (однако, чтобы корректно

## Предобработка данных (2 балла)

In [2]:
import sys
PATH_TO_CLAI_UTILS = "nlp/task_last/clai/utils"
# sys.path.append(PATH_TO_CLAI_UTILS)
sys.path.insert(0, PATH_TO_CLAI_UTILS)

In [322]:
import numpy as np
import pandas as pd

from bashlint.data_tools import bash_parser, pretty_print, cmd2template
from metric.metric_utils import compute_metric
from functools import partial

from collections import Counter
import sentencepiece as spm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

Считаем данные. В столбце `invocation` находится текстовый запрос, в столбце `cmd` находится релевантная команда.

In [486]:
train_data = pd.read_csv('train.csv')
train_data.head()

,invocation,cmd
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`


В тестовых данных столбец `origin` отвечает за источник данных, значения `handrafted` соответствуют парам, составленными людьми, а `mined` парам, собранным автоматически.

In [487]:
test_data = pd.read_csv('test_data.csv')
test_data.head()

,invocation,cmd,origin
0,create ssh connection to specified ip from spe...,ssh user123@176.0.13.154,handcrafted
1,"search for commands containing string ""zeppeli...",history | grep zeppelin,handcrafted
2,search for location of specified file or appli...,whereis python3,handcrafted
3,grant all rights to root folder,sudo chmod 777 -R /,handcrafted
4,search in running processes for specified name,ps -aux | grep zepp,handcrafted


**Задание**. Проведите предобработку текста. Рекомендуется:
* перевести всё в нижний регистр
* удалить стоп-слова (специфичные для выборки)
* провести стемминг токенов
* удалить все символы кроме латинских букв

In [488]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def clean_text(text):
    ### YOUR CODE HERE ###
    text = re.sub(r'[^a-z\s]', ' ', text.lower())
    
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in text.split() if word not in stop_words]
    
    stemmer = SnowballStemmer("english")
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(stemmed_tokens)

nltk.download('stopwords')

In [489]:
train_data['text_cleaned'] = train_data['invocation'].apply(clean_text)
test_data['text_cleaned'] = test_data['invocation'].apply(clean_text)

Для обработки кода воспользуемся функцией `cmd2template`:

In [490]:
train_data['cmd_cleaned'] = train_data['cmd'].apply(partial(cmd2template, loose_constraints=True))
test_data['cmd_cleaned'] = test_data['cmd'].apply(partial(cmd2template, loose_constraints=True))

In [491]:
train_data = train_data.sample(frac=1).reset_index(drop=True)

Разделим данные на обучение и валидацию. Т.к. данных очень мало, то для валидационной выборки выделим только 100 примеров.

In [492]:
valid_data = train_data.iloc[-100:]
train_data = train_data.iloc[:-100]

**Задание**. Стандартный формат входных данных для трансформеров — BPE токены. Воспользуйтесь библиотекой sentencepiece для обучения токенайзеров для текста и кода. Используйте небольшое число токенов.

In [493]:
## YOUR CODE HERE ###

In [494]:
train_data["text_cleaned"].to_csv("text_cleaned.txt", index=False, header=False)
train_data["cmd_cleaned"].to_csv("cmd_cleaned.txt", index=False, header=False)

In [495]:
## YOUR CODE HERE ###

text_tokenizer = spm.SentencePieceTrainer.train(
    input="text_cleaned.txt",
    model_prefix="text_tokenizer",
    vocab_size=2000,
    model_type="bpe",
    pad_id=0,
    unk_id=3,
)

cmd_tokenizer = spm.SentencePieceTrainer.train(
    input="cmd_cleaned.txt",
    model_prefix="cmd_tokenizer",
    vocab_size=500,
    model_type="bpe",
    pad_id=0,
    unk_id=3,
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: text_cleaned.txt
  input_format: 
  model_prefix: text_tokenizer
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  

**Задание**. Задайте датасеты и лоадеры для ваших данных. Каждая последовательность должна начинаться с BOS токена и заканчиваться EOS токеном. Рекомендуется ограничить длину входных и выходных последовательностей!

In [496]:
PAD_ID = 0
BOS_ID = 1
EOS_ID = 2


MAX_TEXT_LENGTH = 256
MAX_CODE_LENGTH = 40

BATCH_SIZE = 64

In [497]:
class TextToBashDataset(Dataset):
    ## YOUR CODE HERE ###
    def __init__(self, corpus_text, corpus_cmd, text_tokenizer, cmd_tokenizer):
        self.text_tokenizer = text_tokenizer
        self.cmd_tokenizer = cmd_tokenizer
        self.corpus_text, self.corpus_cmd = [], []
        for text, cmd in zip(corpus_text, corpus_cmd):
            tokens_text = self.text_tokenizer.encode_as_ids(text)[:MAX_TEXT_LENGTH - 2]
            tokens_cmd = self.cmd_tokenizer.encode_as_ids(cmd)[:MAX_CODE_LENGTH - 2]

            self.corpus_text.append([BOS_ID] + tokens_text + [EOS_ID])
            self.corpus_cmd.append([BOS_ID] + tokens_cmd + [EOS_ID])


    def __len__(self):
        return len(self.corpus_text)

    def __getitem__(self, idx):
        return torch.tensor(self.corpus_text[idx]), torch.tensor(self.corpus_cmd[idx])

In [498]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    texts, cmds = [], []
    for sample in batch:
        text, cmd = sample
        texts.append(text)
        cmds.append(cmd)
    texts = pad_sequence(texts, batch_first=True, padding_value=PAD_ID)
    cmds = pad_sequence(cmds, batch_first=True, padding_value=PAD_ID)
    
    labels = torch.empty_like(cmds)
    labels.copy_(cmds)
    return texts, cmds, labels

In [499]:
## YOUR CODE HERE ###

text_tokenizer = spm.SentencePieceProcessor(model_file="text_tokenizer.model")
cmd_tokenizer = spm.SentencePieceProcessor(model_file="cmd_tokenizer.model")
train_ds = TextToBashDataset(train_data["text_cleaned"].tolist(), train_data["cmd_cleaned"].tolist(), text_tokenizer, cmd_tokenizer)
valid_ds = TextToBashDataset(valid_data["text_cleaned"].tolist(), valid_data["cmd_cleaned"].tolist(), text_tokenizer, cmd_tokenizer)

In [500]:
loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True),
    'valid': DataLoader(valid_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn),
}

## Обучение бейзлайна (2 балла)

In [501]:
from transformers import BertConfig, BertModel, EncoderDecoderConfig, EncoderDecoderModel

**Задание.** Реализуйте модель encoder-decoder ниже. В качестве моделей энкодера и декодера рекомендуется использовать BertModel из библиотеки transformers, заданную через BertConfig. В случае декодера необходимо выставить параметры is_decoder=True и add_cross_attention=True. В качестве модели, <<сцепляющей>> энкодер и декодер, в одну архитектуру рекомендуется использовать EncoderDecoderModel.

**Обратите внимание!** EncoderDecoderModel поддерживает использование кэшированных результатов при последовательной генерации. Это пригодится при реализации beam-search ниже.

Для того, чтобы удобнее задавать модели, рекомендуется реализовать задание модели через конфиг. Ниже представлены базовые параметры, при которых модель должна работать быстро и с приемлемым качеством.

In [549]:
text_model_config = {
    'vocab_size': text_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_hidden_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.25,
    'pad_token_id': PAD_ID,
}

cmd_model_config = {
    'vocab_size': cmd_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_hidden_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.25,
    'pad_token_id': PAD_ID,
    'decoder_start_token_id': BOS_ID,
    'eos_token_id': EOS_ID,
    'is_decoder': True,
    'add_cross_attention': True 
}

In [550]:
class TextToBashModel(nn.Module):
    def __init__(self, text_model_config, cmd_model_config):
        super(TextToBashModel, self).__init__()
        ## YOUR CODE HERE ##
        encoder_config = BertConfig(**text_model_config)
        decoder_config = BertConfig(**cmd_model_config)

        self.config = EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config, decoder_config)
        self.model = EncoderDecoderModel(config=self.config)
        
        self.model.config.decoder_start_token_id = decoder_config.decoder_start_token_id
        self.model.config.pad_token_id = decoder_config.pad_token_id
        self.model.config.eos_token_id = decoder_config.eos_token_id
        self.model.config.vocab_size = cmd_tokenizer.vocab_size()

    def forward(self, input_ids, decoder_input_ids, train=True):
        ## YOUR CODE HERE ##
        if train:
             return self.model(input_ids=input_ids, labels=decoder_input_ids)
        else:
            return self.model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)

    def generate(self, input_ids, attention_mask, num_beams=5, temperature=1.0, max_length=20):
        return self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            num_beams=num_beams,
            temperature=temperature,
            max_length=max_length,
            early_stopping=True
        )

**Задание**. Обучите вашу модель ниже.

Рекомендуется:
* в качестве лосса использовать стандартную кросс-энтропию, не забывайте игнорировать PAD токены
* использовать Adam для оптимизации
* не использовать scheduler для бейзлайна (модель легко переобучается с ним)
* использовать early stopping по валидационному лоссу

In [555]:
## YOUR CODE HERE ##

from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_n

def train(model, criterion, optimizer, dataloaders, n_epochs=20, device='cuda'):
    best_val_loss = float('inf')
    
    for epoch in tqdm_n(range(n_epochs)):
        model.train()
        train_loss = 0.0
        for batch in loaders["train"]:
            input_ids, cmd_ids, _ = batch
            input_ids, cmd_ids = input_ids.to(device), cmd_ids.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(input_ids=input_ids, decoder_input_ids=cmd_ids)
            loss = outputs["loss"]
            # loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
            
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().cpu().item()
        
        avg_train_loss = train_loss / len(loaders["train"])
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in loaders["valid"]:
                input_ids, cmd_ids, _ = batch
                input_ids, cmd_ids = input_ids.to(device), cmd_ids.to(device)
                
                outputs = model(input_ids=input_ids, decoder_input_ids=cmd_ids)
                loss = outputs["loss"]
                # loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
                val_loss += loss.detach().cpu().item()
        
        avg_val_loss = val_loss / len(loaders["valid"])
        print(f"Epoch {epoch} Loss Train/Val: {avg_train_loss:.4f}/{avg_val_loss:.4f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), f"best_model.pt")
        if (epoch+1) % 5 == 0:
            torch.save(model.state_dict(), f"model_epoch_{epoch}.pt")
    return model


In [556]:
model = TextToBashModel(text_model_config, cmd_model_config).to('cuda')
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)

model_trained = train(model, criterion, optimizer, loaders, n_epochs=30, device='cuda')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0 Loss Train/Val: 1.8340/0.7221
Epoch 1 Loss Train/Val: 0.6429/0.5354
Epoch 2 Loss Train/Val: 0.5163/0.4438
Epoch 3 Loss Train/Val: 0.4463/0.4027
Epoch 4 Loss Train/Val: 0.3926/0.3669
Epoch 5 Loss Train/Val: 0.3603/0.3488
Epoch 6 Loss Train/Val: 0.3351/0.3368
Epoch 7 Loss Train/Val: 0.3119/0.3297
Epoch 8 Loss Train/Val: 0.2884/0.3054
Epoch 9 Loss Train/Val: 0.2755/0.2984
Epoch 10 Loss Train/Val: 0.2557/0.3001
Epoch 11 Loss Train/Val: 0.2426/0.3001
Epoch 12 Loss Train/Val: 0.2320/0.2981
Epoch 13 Loss Train/Val: 0.2196/0.2850
Epoch 14 Loss Train/Val: 0.2073/0.2844
Epoch 15 Loss Train/Val: 0.1984/0.2875
Epoch 16 Loss Train/Val: 0.1896/0.2886
Epoch 17 Loss Train/Val: 0.1767/0.2784
Epoch 18 Loss Train/Val: 0.1718/0.2865
Epoch 19 Loss Train/Val: 0.1639/0.2896
Epoch 20 Loss Train/Val: 0.1612/0.2858
Epoch 21 Loss Train/Val: 0.1520/0.2752
Epoch 22 Loss Train/Val: 0.1473/0.2812
Epoch 23 Loss Train/Val: 0.1401/0.2744
Epoch 24 Loss Train/Val: 0.1354/0.2817
Epoch 25 Loss Train/Val: 0.1297/0.2

In [267]:
torch.save(model_trained.state_dict(), "model.pt")

## Генерация команд (2 балла)

**Задание**. Реализуйте алгоритм beam-search в классе BeamSearchGenerator ниже. Ваша реализация должна поддерживать задание температуры софтмакса. Выходы модели, полученные на предыдущих итерациях, необходимо кэшировать для повышения скорости алгоритма. Вместо подсчёта произведения любых вероятностей необходимо считать сумму их логарифмов.

Алгоритм должен возвращать список пар из получившихся выходных последовательностей и логарифмов их вероятностей.

In [536]:
class BeamSearchGenerator:
    def __init__(
            self, pad_id, eos_id, bos_id,
            max_length=20, beam_width=5, temperature=1,
            device='cuda',
    ):
        """
        Parameters
        ----------
        pad_id : int
        eos_id : int
        bos_id : int
        max_length : int
            Maximum length of output sequence
        beam_width : int
            Width of the beam
        temperature : float
            Softmax temperature
        device : torch.device
            Your model device
        """
        self.pad_id = pad_id
        self.eos_id = eos_id
        self.bos_id = bos_id

        self.max_length = max_length
        self.beam_width = beam_width
        self.temperature = temperature

        self.device = device

    def get_result_generate(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        """
        ## YOUR CODE HERE ##
        attention_mask = (input_text_tokens != self.pad_id).float()
        return [model.generate(input_text_tokens, attention_mask, num_beams=self.beam_width,
                              temperature=self.temperature, max_length=self.max_length) for _ in range(self.beam_width)]


    def get_result(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        """
        ## YOUR CODE HERE ##
        attention_mask = (input_text_tokens != self.pad_id).float()
        
        generated_ids = [(torch.tensor([self.bos_id], device=self.device), 0.0)]
        cache = {}

        for _ in range(self.max_length):
            all_candidates = []

            for generated_seq, log_prob in generated_ids:
                if generated_seq[-1].item() == self.eos_id:
                    all_candidates.append((generated_seq, log_prob))
                    continue

                seq_key = tuple(generated_seq.tolist())
                if seq_key in cache:
                    logits = cache[seq_key]
                else:
                    decoder_attention_mask = (generated_seq != self.pad_id).float().unsqueeze(0)
                    logits = model(input_text_tokens, generated_seq.unsqueeze(0), train=False)
                    logits = logits["logits"].squeeze(0)
                    cache[seq_key] = logits

                logits = logits / self.temperature
                probs = F.log_softmax(logits[-1], dim=-1)
                for tok_id in range(probs.size(0)):
                    new_seq = torch.cat([generated_seq, torch.tensor([tok_id], device=self.device)])
                    new_log_prob = log_prob + probs[tok_id].item()
                    all_candidates.append((new_seq, new_log_prob))

            generated_ids = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:self.beam_width]
            if all(gen_seq[-1].item() == self.eos_id for gen_seq, _ in generated_ids):
                break
        return [(gen_seq.tolist(), log_prob) for gen_seq, log_prob in generated_ids]

Протестируйте на нескольких примерах работу вашего алгоритма. Если всё реализовано правильно, то как минимум на трёх примерах из 5 всё должно работать правильно.

In [537]:
beam_search_engine = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=10,
    temperature=1, device='cuda',
)

In [558]:
# model = TextToBashModel(text_model_config, cmd_model_config)
# model.load_state_dict(torch.load("model_epoch_19.pt"))
# model = model.to("cuda")

In [516]:
with torch.no_grad():
    for i in range(5):
        print()
        print('text:', valid_data.invocation.iloc[i])
        print('true:', valid_data.cmd.iloc[i])
        print('true cleaned:', valid_data.cmd_cleaned.iloc[i])

        src = torch.tensor(text_tokenizer.encode(valid_data.text_cleaned.tolist()[i])).to("cuda").unsqueeze(0)
        pred = beam_search_engine.get_result_generate(model, src)
        score = compute_metric(pred, 1, valid_data.cmd.iloc[i])
        # print("default generate: ", cmd_tokenizer.decode(pred.tolist())[0], score)

        pred = beam_search_engine.get_result(model, src)
        scores = []
        for x, proba in pred:
            pred_cmd = cmd_tokenizer.decode(list(map(int, x)))
            score = compute_metric(pred_cmd, 1, valid_data.cmd.iloc[i])
            scores.append(score)
            print(pred_cmd, proba)
        print(max(scores))


text: write "deb blah ... blah" to standard output and append to "/etc/apt/sources.list" as root
true: echo 'deb blah ... blah' | sudo tee --append /etc/apt/sources.list
true cleaned: echo Regex | tee --append File
echo Regex | tee -a File -0.8335579037666321
echo Regex | tee File -1.3231375217437744
cat File | tee File -3.460297107696533
echo -e Regex | tee -a File -4.422310829162598
echo Regex Regex | tee -a File -4.563908100128174
cat File | tee -a File -4.879418849945068
echo Regex | tee -a File File -4.90671443939209
mv File File -4.956876277923584
cut -d Regex -f Number File -5.345022201538086
cut -d Regex -f Number | tee File -5.722093105316162
0.5

text: find all the files in the entire filesystem which belong to the group root and display the ten files.
true: find / -group root | head
true cleaned: find Path -group Regex | head
find Path -group Regex | head -0.880927562713623
find Path -group Regex -perm -Permission | head -1.5331900119781494
find Path -group Regex -perm -Per

**Задание**. Дополните функцию для подсчёта качества. Посчитайте качество вашей модели на валидационном и тестовых датасетов.

In [510]:
def compute_all_scores(model, df, beam_engine):
    all_scores = []

    for text, target_cmd in tqdm(zip(df.text_cleaned.values, df.cmd.values), total=len(df.cmd.values)):
        ## YOUR CODE HERE ##
        input_tokens = torch.tensor([BOS_ID] + text_tokenizer.encode_as_ids(text) + [EOS_ID]).to("cuda").unsqueeze(0)
        predictions = beam_engine.get_result(model, input_tokens)

        # get only 5 top results
        predictions = predictions[:5]
        object_scores = []
        for output_tokens, proba in predictions:
            output_cmd = cmd_tokenizer.decode(list(map(int, output_tokens)))
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)

        all_scores.append(max(object_scores))
    return all_scores

In [514]:
def compute_all_scores_generate(model, df, beam_engine):
    all_scores = []

    for text, target_cmd in tqdm(zip(df.text_cleaned.values, df.cmd.values), total=len(df.cmd.values)):
        ## YOUR CODE HERE ##
        input_tokens = torch.tensor([BOS_ID] + text_tokenizer.encode_as_ids(text) + [EOS_ID]).to("cuda").unsqueeze(0)
        predictions = beam_engine.get_result_generate(model, input_tokens)

        predictions = predictions[:5]
        object_scores = []
        for output_tokens in predictions:
            output_cmd = cmd_tokenizer.decode(output_tokens.tolist())
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)

        all_scores.append(max(object_scores))
    return all_scores

Ваша цель при помощи подбора параметров модели и генерации получить средний скор на валидации >= 0.2, скор `handcrafted` части теста >= 0.1. На `mined` части датасета скор может быть низкий, т.к. некоторых команд из датасета нет в обучении.

**Обратите внимание.** Так как датасет для обучения не очень большой, а данные достаточно нестабильные, подбор параметров может очень сильно влиять на модель. Некоторые полезные советы:
* Отслеживайте качество модели после каждой эпохи, не забывайте про early stopping
* Вы можете сразу приступить к следующей части. Побитие скора в этой части задания при помощи трюков из бонусной части считается валидным.

In [436]:
## YOUR CODE HERE ##

In [512]:
all_scores = compute_all_scores(model, valid_data, beam_search_engine)
sum(all_scores) / len(all_scores)

100%|██████████| 100/100 [00:52<00:00,  1.89it/s]


0.4288035714285715

In [511]:
handcrafted = compute_all_scores(model, test_data[test_data["origin"] == "handcrafted"], beam_search_engine)
sum(handcrafted) / len(handcrafted)

100%|██████████| 129/129 [01:01<00:00,  2.08it/s]


0.16147717484926788

In [513]:
mined = compute_all_scores(model, test_data[test_data["origin"] == "mined"], beam_search_engine)
sum(mined) / len(mined)

100%|██████████| 592/592 [04:27<00:00,  2.22it/s]


-0.32590726887601895

## Улучшение модели (4 балла)

Вы реализовали бейзлайн, пришло время улучшить качество модели. Т.к. это последнее задание, мы не будем предлагать конкретные шаги, а только дадим несколько советов.

1. Большой источник информации о работе командной строке — её документация, man. Один из способов улучшения модели - использование мана для генерации новых примеров. Структурированный ман можно найти по ссылке https://github.com/IBM/clai/blob/nlc2cmd/docs/manpage-data.md.
2. Ещё один способ улучшить модель, разделить предсказание утилит и флагов. Т.к. задача предсказания утилит более важная, вы можете натренировать модель, которая предсказывает последовательность утилит, а затем к каждой утилите генерировать флаги.
3. Можно аугментировать данные, чтобы увеличить выборку.
4. Можно в качество входа подавать не только текстовый запрос, но и описание из мана. Т.к. всё описание достаточно большое, нужно сделать дополнительную модель, которая будет выбирать команды, для которых нужно вытащить описание.
5. Найти дополнительные данные, улучшающие обучение
6. Как всегда можно просто сделать больше слоёв, увеличить размер скрытого слоя и т.д.

От вас ожидается скор на валидации >= 0.25, `mined` >= 0, `handrafted` >= 0.15.

In [ ]:
## YOUR CODE HERE ##

## Бонусные баллы (до 3 баллов)

При существенном улучшении качества будут назначаться бонусные баллы. На тестовых датасетах реально выбить качество >= 0.3 на каждом, но усилий потребуется немало...